# Lesson 1 - Building a QA Agent with Google Gemini

```mermaid
graph LR
    %% User / Client Layer
    User([User / A2A Client])
    
    %% Main Orchestrator Layer (Lesson 8)
    subgraph OrchestratorLayer [Router/Requirement Agent]
        Concierge["<b>Healthcare Concierge Agent</b><br/>(BeeAI Framework)<br/><code>Port: 9996</code>"]
    end

    subgraph SubAgents [A2A Agent Servers]
        direction TB

        PolicyAgent["<b>Policy Agent</b><br/>(Gemini with A2A SDK)<br/><code>Port: 9999</code>"]
        ResearchAgent["<b>Research Agent</b><br/>(Google ADK)<br/><code>Port: 9998</code>"]

        ProviderAgent["<b>Provider Agent</b><br/>(LangGraph + LangChain)<br/><code>Port: 9997</code>"]
    end

    %% Data & Tools Layer
    subgraph DataLayer [Data Sources & Tools]
        PDF["Policy PDF"]
        Google[Google Search Tool]
        MCPServer["FastMCP Server<br/>(<code>doctors.json</code>)"]
    end
    
    Label_UA["Sends Query - A2A"]
    Label_CP["A2A"]
    Label_CR["A2A"]
    Label_CProv["A2A"]
    Label_MCP["MCP (stdio)"]

    %% -- CONNECTIONS --
    
    User --- Label_UA --> Concierge

    Concierge --- Label_CP --> PolicyAgent
    Concierge --- Label_CR --> ResearchAgent
    Concierge --- Label_CProv --> ProviderAgent
    
    PolicyAgent -- "Reads" --> PDF
    ResearchAgent -- "Calls" --> Google
    
    ProviderAgent --- Label_MCP --> MCPServer

    classDef orchestrator fill:#f9f,stroke:#333,stroke-width:2px;
    classDef agent fill:#e1f5fe,stroke:#0277bd,stroke-width:2px;
    classDef tool fill:#fff3e0,stroke:#ef6c00,stroke-width:1px,stroke-dasharray: 5 5;
    
    classDef protocolLabel fill:#ffffff,stroke:none,color:#000;
    
    class Concierge orchestrator;
    class PolicyAgent,ResearchAgent,ProviderAgent agent;
    class PDF,Google,MCPServer tool;
    
    class Label_UA,Label_CP,Label_CR,Label_CProv,Label_MCP protocolLabel;
```

In this lesson, you will build a basic Question Answering (QA) agent using [Google Gemini](https://deepmind.google/models/gemini/) via the [Google Gen AI SDK](https://github.com/googleapis/python-genai). You will use **[LiteLLM](https://www.litellm.ai/)** to interact with the model to analyze a PDF document containing health insurance policy details. Then, you will refactor this logic into a reusable Python class, laying the groundwork for the next lesson where you will wrap this agent in an [Agent2Agent (A2A)](https://a2a-protocol.org/) server.

## 1.1. Import Libraries and Setup

First, import the necessary libraries. You will use `litellm` to interact with the LLM and standard libraries to handle file encoding.

In [1]:
import base64
from pathlib import Path

import litellm
from IPython.display import Markdown, display

from helpers import setup_env

In [2]:
setup_env()

## 1.2. Load and Encode Data

You read the insurance policy PDF (`2026AnthemgHIPSBC.pdf`) and encode it in base64 so it can be passed to the model as context.


In [4]:
with Path("data/2026AnthemgHIPSBC.pdf").open("rb") as file:
    pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

## 1.3. Query the Model

Now you will send a specific query to the model: "How much would I pay for mental health therapy?". You provide the model with a system instruction to act as an expert insurance agent and pass the PDF document alongside the user's text prompt.


In [5]:
prompt = "How much would I pay for mental health therapy?"

In [6]:
response = litellm.completion(
    model="gemini/gemini-3-flash-preview",
    # For Vertex AI:
    # model="vertex_ai/gemini-3-flash-preview",
    reasoning_effort="minimal",
    max_tokens=1000,
    messages=[
        {
            "role": "system",
            "content": "You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'",
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:application/pdf;base64,{pdf_data}"},
                },
            ],
        },
    ],
)

In [7]:
response_text = response.choices[0].message.content.replace("$", r"\\$")
display(Markdown(response_text))

For mental health outpatient office visits, you would pay:

*   **In-Network Provider:** 10% coinsurance after your deductible is met.
*   **Out-of-Network Provider:** 30% coinsurance after your deductible is met.

The overall deductible for this plan is **\\$1,700** for an individual or **\\$3,400** for a family when using in-network providers. All coinsurance costs apply only after these deductible amounts have been reached.

## 1.4. Refactor into an Agent Class

To make this code reusable and easier to integrate into an A2A server later, we have wrapped the logic into a `PolicyAgent` class in a file named `policy_agent.py`. This class initializes the data in the `__init__` method and exposes an `answer_query` method.

In [8]:
from IPython.display import Code, display

display(Code("policy_agent.py"))

"""
Core Agent Logic (The "Brains" of the Agent)
--------------------------------------------
As a Data Analyst, you are used to writing SQL or Python to query a structured database.
This file does something similar, but instead of querying a database, it queries an AI 
model (an LLM) using an unstructured document (a PDF file).

While `a2a_policy_agent.py` handles the networking (making this agent available 
to other agents on the internet), this file (`policy_agent.py`) contains the actual 
logic for reading a document and answering questions about it.
"""

import base64
from pathlib import Path

# litellm is a Python library that acts like a universal translator for AI models.
# It lets you write code once, and easily switch between different AI models 
# (like Google's Gemini, OpenAI's GPT-4, or Anthropic's Claude) just by changing one line of code.
import litellm

# A helper function to load API keys and other settings from a .env file
from helpers import setup_env


class PolicyAgent:
    def __init__(self) -> None:
        """
        Think of this __init__ method like your 'data setup' phase before you run an analysis.
        This runs only once when the agent is first started.
        """
        setup_env() # Load API keys
        
        # 1. READ THE FILE: We open the health insurance policy PDF file.
        # "rb" means "read binary". We are reading the raw bytes of the PDF.
        with Path("data/2026AnthemgHIPSBC.pdf").open("rb") as file:
            
            # 2. ENCODE THE FILE: We can't easily send raw files over an internet API request to an AI.
            # So, we convert the raw PDF file into a long string of text using 'base64' encoding.
            # This 'self.pdf_data' variable now holds the entire PDF document in a format the AI can read over the web.
            self.pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

    def answer_query(self, prompt: str) -> str:
        """
        This is the main function that gets called whenever someone asks the agent a question.
        The 'prompt' variable contains the text of the user's question (e.g., "What is the deductible?").
        """
        
        # 3. ASK THE AI: We make a request to the AI model using litellm.
        response = litellm.completion(
            # We are telling it to use Google's 'Gemini 3 Flash' model.
            # This specific model is "multimodal", meaning it's really good at reading files (like PDFs).
            model="gemini/gemini-3-flash-preview",
            
            # For Vertex AI
            # model="vertex_ai/gemini-3-flash-preview",
            
            # These settings tell the AI not to overthink the answer and to keep it relatively short (under 1000 tokens).
            reasoning_effort="minimal",
            max_tokens=1000,
            
            # 4. THE MESSAGE: The 'messages' list is how we talk to the AI.
            messages=[
                {
                    # A 'system' role is used to give the AI its permanent instructions and personality.
                    # Think of this as the business rules for your data analysis.
                    "role": "system",
                    "content": "You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'",
                },
                {
                    # A 'user' role is the actual request we are making right now.
                    "role": "user",
                    "content": [
                        # First, we pass the user's actual question...
                        {"type": "text", "text": prompt},
                        
                        # Second, we pass the PDF document we loaded earlier!
                        # This allows the AI to "read" the PDF to find the answer.
                        {
                        

## 1.5. Test the Agent Class

Finally, import the `PolicyAgent` class you just created and test it with the same query to ensure it works as expected.

In [9]:
from policy_agent import PolicyAgent

print("Running Health Insurance Policy Agent")
agent = PolicyAgent()
prompt = "How much would I pay for mental health therapy?"

response = agent.answer_query(prompt)
display(Markdown(response))

Running Health Insurance Policy Agent


Under the gHIP HDHP plan, your costs for mental health therapy depend on whether the provider is in-network and if you have met your deductible:

*   **In-Network Provider:** You pay **10% coinsurance** after your deductible is met.
*   **Out-of-Network Provider:** You pay **30% coinsurance** after your deductible is met.

**Important Deductible Information:**
Before the plan begins to pay these coinsurance percentages, you must meet the overall annual deductible:
*   **In-Network:** \$1,700 for individuals / \$3,400 for families.
*   **Out-of-Network:** \$3,400 for individuals / \$6,800 for families.

These rates apply to both outpatient office visits and other outpatient mental health services. For inpatient mental health services, the same 10% (in-network) or 30% (out-of-network) coinsurance applies to both the facility fee and the physician fee.

## 1.6. Resources

- [Google Gemini API Documentation](https://ai.google.dev/docs)
- [LiteLLM Documentation](https://docs.litellm.ai/docs/)